In [10]:
# create a list of tuple with the token and the time-step
print_order = ['안녕하세요.', '저는', '오늘', '서울', '에', '왔어요.', '앞으로', '잘', '부탁드립니다.']
sequence = [{print_order[i]} for i in range(len(print_order))]
sequence

[{'안녕하세요.'},
 {'저는'},
 {'오늘'},
 {'서울'},
 {'에'},
 {'왔어요.'},
 {'앞으로'},
 {'잘'},
 {'부탁드립니다.'}]

In [11]:
sliding_winsow_size = 3

def sliding_window_attention(seq: list[set[str]], w: int):
    seq_len = len(seq)
    attetion_scores: list[list[set]] = [[None for _ in range(seq_len)] for _ in range(seq_len)]
    for i, q_token_set in enumerate(seq):
        for j, k_token_set in enumerate(seq):
            # The upper tiangle is all None
            if j > i:
                continue
            if i - j >= w:
                continue

            attention = set()
            # Add all tokens from q_tokens to attention_result
            attention.update(q_token_set)
            # Add all tokens from k_tokens to attention_result
            attention.update(k_token_set)

            attetion_scores[i][j] = attention
    return attetion_scores

def multiple_by_v(attention_scores: list[list[set]], v_sequence: list[set[str]]) -> list[set[str]]:
    seq_len = len(v_sequence)
    result = [set() for _ in range(seq_len)]
    for i in range(seq_len):
        for j in range(seq_len):
            attention = attention_scores[i][j]
            v = v_sequence[j]
            r = result[i]
            # Add all the tokens in the attention (if not None) to r
            if attention is not None:
                # Add all the tokens in v to r
                r.update(v)
                r.update(attention)
    return result

def print_attention(attention_scores: list[list[set[str]]]):
    for i, row in enumerate(attention_scores):
        for j, attention in enumerate(row):
            if attention is None:
                print('None', end='\t')
            else:
                print(f'{sorted(attention, key=lambda x: print_order.index(x))}', end='\t')
        print()

def print_sequence(seq: list[set[str]]):
    for i, token_set in enumerate(seq):
        print(f'{i}: {sorted(token_set, key=lambda x: print_order.index(x))}')

def print_layer(input: list[set[str]], layer_num: int) -> list[set[str]]:
    print(f'Layer {layer_num} input:')
    print_sequence(input)
    attention_scores = sliding_window_attention(input, sliding_winsow_size)
    print()
    print(f'Layer {layer_num} attention_scores:')
    print_attention(attention_scores)
    output = multiple_by_v(attention_scores, input)
    print()
    print(f'Layer {layer_num} output:')
    print_sequence(output)
    return output

In [12]:
# Layer 1
output_layer_1 = print_layer(sequence, 1)

Layer 1 input:
0: ['안녕하세요.']
1: ['저는']
2: ['오늘']
3: ['서울']
4: ['에']
5: ['왔어요.']
6: ['앞으로']
7: ['잘']
8: ['부탁드립니다.']

Layer 1 attention_scores:
['안녕하세요.']	None	None	None	None	None	None	None	None	
['안녕하세요.', '저는']	['저는']	None	None	None	None	None	None	None	
['안녕하세요.', '오늘']	['저는', '오늘']	['오늘']	None	None	None	None	None	None	
None	['저는', '서울']	['오늘', '서울']	['서울']	None	None	None	None	None	
None	None	['오늘', '에']	['서울', '에']	['에']	None	None	None	None	
None	None	None	['서울', '왔어요.']	['에', '왔어요.']	['왔어요.']	None	None	None	
None	None	None	None	['에', '앞으로']	['왔어요.', '앞으로']	['앞으로']	None	None	
None	None	None	None	None	['왔어요.', '잘']	['앞으로', '잘']	['잘']	None	
None	None	None	None	None	None	['앞으로', '부탁드립니다.']	['잘', '부탁드립니다.']	['부탁드립니다.']	

Layer 1 output:
0: ['안녕하세요.']
1: ['안녕하세요.', '저는']
2: ['안녕하세요.', '저는', '오늘']
3: ['저는', '오늘', '서울']
4: ['오늘', '서울', '에']
5: ['서울', '에', '왔어요.']
6: ['에', '왔어요.', '앞으로']
7: ['왔어요.', '앞으로', '잘']
8: ['앞으로', '잘', '부탁드립니다.']


In [13]:
# Layer 2
output_layer_2 = print_layer(output_layer_1, 2)

Layer 2 input:
0: ['안녕하세요.']
1: ['안녕하세요.', '저는']
2: ['안녕하세요.', '저는', '오늘']
3: ['저는', '오늘', '서울']
4: ['오늘', '서울', '에']
5: ['서울', '에', '왔어요.']
6: ['에', '왔어요.', '앞으로']
7: ['왔어요.', '앞으로', '잘']
8: ['앞으로', '잘', '부탁드립니다.']

Layer 2 attention_scores:
['안녕하세요.']	None	None	None	None	None	None	None	None	
['안녕하세요.', '저는']	['안녕하세요.', '저는']	None	None	None	None	None	None	None	
['안녕하세요.', '저는', '오늘']	['안녕하세요.', '저는', '오늘']	['안녕하세요.', '저는', '오늘']	None	None	None	None	None	None	
None	['안녕하세요.', '저는', '오늘', '서울']	['안녕하세요.', '저는', '오늘', '서울']	['저는', '오늘', '서울']	None	None	None	None	None	
None	None	['안녕하세요.', '저는', '오늘', '서울', '에']	['저는', '오늘', '서울', '에']	['오늘', '서울', '에']	None	None	None	None	
None	None	None	['저는', '오늘', '서울', '에', '왔어요.']	['오늘', '서울', '에', '왔어요.']	['서울', '에', '왔어요.']	None	None	None	
None	None	None	None	['오늘', '서울', '에', '왔어요.', '앞으로']	['서울', '에', '왔어요.', '앞으로']	['에', '왔어요.', '앞으로']	None	None	
None	None	None	None	None	['서울', '에', '왔어요.', '앞으로', '잘']	['에', '왔어요.', '앞으로', '잘']	['왔어요.', '앞으로', 

In [14]:
# Layer 3
output_layer_3 = print_layer(output_layer_2, 3)

Layer 3 input:
0: ['안녕하세요.']
1: ['안녕하세요.', '저는']
2: ['안녕하세요.', '저는', '오늘']
3: ['안녕하세요.', '저는', '오늘', '서울']
4: ['안녕하세요.', '저는', '오늘', '서울', '에']
5: ['저는', '오늘', '서울', '에', '왔어요.']
6: ['오늘', '서울', '에', '왔어요.', '앞으로']
7: ['서울', '에', '왔어요.', '앞으로', '잘']
8: ['에', '왔어요.', '앞으로', '잘', '부탁드립니다.']

Layer 3 attention_scores:
['안녕하세요.']	None	None	None	None	None	None	None	None	
['안녕하세요.', '저는']	['안녕하세요.', '저는']	None	None	None	None	None	None	None	
['안녕하세요.', '저는', '오늘']	['안녕하세요.', '저는', '오늘']	['안녕하세요.', '저는', '오늘']	None	None	None	None	None	None	
None	['안녕하세요.', '저는', '오늘', '서울']	['안녕하세요.', '저는', '오늘', '서울']	['안녕하세요.', '저는', '오늘', '서울']	None	None	None	None	None	
None	None	['안녕하세요.', '저는', '오늘', '서울', '에']	['안녕하세요.', '저는', '오늘', '서울', '에']	['안녕하세요.', '저는', '오늘', '서울', '에']	None	None	None	None	
None	None	None	['안녕하세요.', '저는', '오늘', '서울', '에', '왔어요.']	['안녕하세요.', '저는', '오늘', '서울', '에', '왔어요.']	['저는', '오늘', '서울', '에', '왔어요.']	None	None	None	
None	None	None	None	['안녕하세요.', '저는', '오늘', '서울', '에', '왔어요.', '

In [15]:
# Layer 4
output_layer_4 = print_layer(output_layer_3, 4)

Layer 4 input:
0: ['안녕하세요.']
1: ['안녕하세요.', '저는']
2: ['안녕하세요.', '저는', '오늘']
3: ['안녕하세요.', '저는', '오늘', '서울']
4: ['안녕하세요.', '저는', '오늘', '서울', '에']
5: ['안녕하세요.', '저는', '오늘', '서울', '에', '왔어요.']
6: ['안녕하세요.', '저는', '오늘', '서울', '에', '왔어요.', '앞으로']
7: ['저는', '오늘', '서울', '에', '왔어요.', '앞으로', '잘']
8: ['오늘', '서울', '에', '왔어요.', '앞으로', '잘', '부탁드립니다.']

Layer 4 attention_scores:
['안녕하세요.']	None	None	None	None	None	None	None	None	
['안녕하세요.', '저는']	['안녕하세요.', '저는']	None	None	None	None	None	None	None	
['안녕하세요.', '저는', '오늘']	['안녕하세요.', '저는', '오늘']	['안녕하세요.', '저는', '오늘']	None	None	None	None	None	None	
None	['안녕하세요.', '저는', '오늘', '서울']	['안녕하세요.', '저는', '오늘', '서울']	['안녕하세요.', '저는', '오늘', '서울']	None	None	None	None	None	
None	None	['안녕하세요.', '저는', '오늘', '서울', '에']	['안녕하세요.', '저는', '오늘', '서울', '에']	['안녕하세요.', '저는', '오늘', '서울', '에']	None	None	None	None	
None	None	None	['안녕하세요.', '저는', '오늘', '서울', '에', '왔어요.']	['안녕하세요.', '저는', '오늘', '서울', '에', '왔어요.']	['안녕하세요.', '저는', '오늘', '서울', '에', '왔어요.']	None	None	None	
Non

In [16]:
# Layer 5
output_layer_5 = print_layer(output_layer_4, 5)

Layer 5 input:
0: ['안녕하세요.']
1: ['안녕하세요.', '저는']
2: ['안녕하세요.', '저는', '오늘']
3: ['안녕하세요.', '저는', '오늘', '서울']
4: ['안녕하세요.', '저는', '오늘', '서울', '에']
5: ['안녕하세요.', '저는', '오늘', '서울', '에', '왔어요.']
6: ['안녕하세요.', '저는', '오늘', '서울', '에', '왔어요.', '앞으로']
7: ['안녕하세요.', '저는', '오늘', '서울', '에', '왔어요.', '앞으로', '잘']
8: ['안녕하세요.', '저는', '오늘', '서울', '에', '왔어요.', '앞으로', '잘', '부탁드립니다.']

Layer 5 attention_scores:
['안녕하세요.']	None	None	None	None	None	None	None	None	
['안녕하세요.', '저는']	['안녕하세요.', '저는']	None	None	None	None	None	None	None	
['안녕하세요.', '저는', '오늘']	['안녕하세요.', '저는', '오늘']	['안녕하세요.', '저는', '오늘']	None	None	None	None	None	None	
None	['안녕하세요.', '저는', '오늘', '서울']	['안녕하세요.', '저는', '오늘', '서울']	['안녕하세요.', '저는', '오늘', '서울']	None	None	None	None	None	
None	None	['안녕하세요.', '저는', '오늘', '서울', '에']	['안녕하세요.', '저는', '오늘', '서울', '에']	['안녕하세요.', '저는', '오늘', '서울', '에']	None	None	None	None	
None	None	None	['안녕하세요.', '저는', '오늘', '서울', '에', '왔어요.']	['안녕하세요.', '저는', '오늘', '서울', '에', '왔어요.']	['안녕하세요.', '저는', '오늘', '서울', '에', '